###Ingest lap_times json files

In [0]:
dbutils.widgets.text('p_data_source','')
v_data_source = dbutils.widgets.get('p_data_source')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#####Step 1 - Read the JSON file using the spark dataframe reader API

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [0]:
lap_times_schema = StructType(fields = [StructField('raceId',IntegerType(),False),
                                      StructField('driverId',IntegerType(),True),
                                      StructField('lap',IntegerType(),True),
                                      StructField('position',IntegerType(),True),
                                      StructField('time',StringType(),True),
                                      StructField('milliseconds',IntegerType(),True)])

In [0]:
lap_times_df = spark.read.csv(f"{raw_folder_path}/lap_times",schema = lap_times_schema)

#####Step 2 - Rename columns and add new columns

In [0]:
from pyspark.sql.functions import current_timestamp,lit

In [0]:
lap_times_final_df = lap_times_df.withColumnRenamed('driverId','driver_id') \
        .withColumnRenamed('raceId','race_id') \
        .withColumn('ingestion_date',current_timestamp()) \
        .withColumn('data_source',lit(v_data_source))

#####Step 3 - Write the output to processed container in parquet format

In [0]:
lap_times_final_df.write.mode('overwrite').parquet(f"{processed_folder_path}/lap_times")

In [0]:
dbutils.notebook.exit('Success')